In [ ]:
import os
import csv
import math
import librosa
import librosa.display
import numpy as np
import pandas as pd
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from python_speech_features import mfcc
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import kurtosis, skew

In [ ]:
# load training data.
rootLocation = "data/genres" # directory which contains the folder of genres. 
genres = os.listdir(rootLocation)
genres.remove('.DS_Store') if '.DS_Store' in genres else None
data = list() # read .au files from librosa library
labels = list() # list of labels

for genre in genres:
    clips = os.listdir(rootLocation+"/{}".format(genre))
    clips.remove('.DS_Store') if '.DS_Store' in clips else None
    
    for clip in clips: # using first 2 songs from each genre. remove [:2] to loop through all songs.
        musicLabel = clip.split('.',1)[0]
        musicData = librosa.load('data/genres/{}/{}'.format(genre,clip))
        #musicData = rootLocation + "/{}/{}".format(genre,clip)
        
        labels.append(musicLabel)
        data.append(musicData)
        

In [ ]:
# Load Validation data:
v_Location = "data/validation" # directory which contains the folder of genres. 
v_clips = os.listdir(v_Location)
v_clips.array('.DS_Store') if '.DS_Store' in v_clips else None
v_data = list() # read .au files from librosa library  
v_label = list()
for v_clip in v_clips:
    v_musicData = librosa.core.load(v_Location+'/{}'.format(v_clip))
    #musicData = rootLocation+"/{}/{}".format(genre,clip)
    v_label.append(v_clip)
    v_data.append(v_musicData)

In [ ]:
# plot spectrogram from the data read
c = np.abs(librosa.stft(data[40][0]))
librosa.display.specshow(librosa.power_to_db(c**2, ref=np.max),sr=data[2][1], y_axis='log', x_axis='time')
plt.colorbar()
plt.show()

In [ ]:
# play the selected audio
Audio(data = v_data[71][0], rate = v_data[71][1])

In [ ]:
# First Feature ex
# fast Fourier Transform feature extraction
def fourierfeatures(data, n = 1000): # data in format [[y], sample_rate]
    fft_features= []
    for clip in data:
        features = abs(fft(clip[0]))[:n]
        m = max(features)
        norm_features = [x / m for x in features]
        fft_features.append(norm_features) # normalize the numbers between 0 and 1.
    return fft_features

In [ ]:
# MFCC feature extraction
def mfccfeatures(data): # data in format [[y], sample_rate]
    mfcc_features= []
    for clip in data:
        features= mfcc(clip[0])
        avg_f = np.mean(features, axis=0)
        m = max(avg_f)
        norm_features = [x / m for x in avg_f]
        mfcc_features.append(norm_features) 
    return mfcc_features

In [ ]:
# Custom Feature extraction + MFCC feature extraction
def customfeatures(data): # data in format [[y], sample_rate]
    mfcc_features= []
    for clip in data:
        ## mfcc features as the base. 
        features= mfcc(clip[0],samplerate = clip[1], nftt = 600)
        avg_f = np.mean(features, axis=0)
        
        ## add tempo value as the 14th feature
        tempo = [a/15 for a in librosa.beat.tempo(y=clip[0], sr=clip[1])]
        avg_f= np.append(avg_f, tempo)  # add tempo as 14th featur
        
        ## normalize between 0 and 1
        #m = max(abs(avg_f))
        #norm_features = [x/m for x in avg_f]
        mfcc_features.append(avg_f) 
    return mfcc_features

In [ ]:
def getBest_Features(data):
    i = 0
    best_features= list()
    for clip in data:
        features= mfcc(clip[0])
        avg_f = np.mean(features, axis=0)
        std_f = np.std(features,axis=0)
        skew_f = skew (features, axis =0)
        kurt_f = kurtosis(features, axis =0)
        addList = np.concatenate((avg_f,std_f,skew_f,kurt_f))
        best_features.append(addList)
        i +=1
        print(i)
    return best_features

In [ ]:
rhythm= librosa.feature.tempogram(data[0][0], sr = data[0][1])
avg_f = np.mean(rhythm, axis=0)

#print(pd.DataFrame(avg_f))



In [ ]:
training_features = getBest_Features(data = data)

In [ ]:
validating_features = getBest_Features(data = v_data)

In [ ]:
print(pd.DataFrame(training_features))

In [ ]:
#COnvert to Pandas dataframe.. better compatibility with the algorithms
tdf = pd.DataFrame(training_features)
vdf = pd.DataFrame(validating_features)
ldf = pd.DataFrame(labels)

In [ ]:
# encapsulate other ML libraries
def genericML1(trainingdata,traininglabels):
    model = RandomForestClassifier(n_estimators= 800) # constructor for ML class
    model.fit(trainingdata, traininglabels)
    return model

In [ ]:
# encapsulate other ML libraries
def genericML2(trainingdata,traininglabels):
    model = MLPClassifier(max_iter = 1000, random_state =2) # constructor for Ml class
    model.fit(trainingdata,traininglabels)
    return model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tdf, ldf, test_size=0.2, random_state=32)

In [ ]:
mlObj = genericML1(tdf,ldf.values.ravel())

In [ ]:
prediction = mlObj.predict(vdf)

In [ ]:
# make prediction
prediction = mlObj.predict(vdf)
#actual = list(y_test.values)
#print(list(actual.values))
count=0
for p,a in zip(prediction,actual):
    if p == a: count += 1 
print(count/len(prediction))
#print(y_test)
    
#print(v_label)

In [ ]:
# write to csv file
with open('predict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile,lineterminator='\n')
    writer.writerow(['id', 'class'])
    for file , p_label in zip(v_label, prediction):
        writer.writerow([str(file),str(p_label)])
    

In [ ]:
print(prediction)